In [1]:
import sys
sys.path.append('../')
import exp
import regression as r

In [2]:
df = exp.get_wp_energy_data()
df.head()

,PET,VPD,air_temp,doy,precip,soil_temp,sw_in,wind_speed,year,wp_RNET,...,wp_bnd3,wp_bnd7,wp_evi,wp_lswi2,wp_ndvi,wp_LST.day,wp_LST.night,LW_IN.wp,LW_IN.si,LW_IN
0,7.33,0.808731,19.179167,195,0.0,22.320833,30.3156,4.958333,2012,20.798342,...,0.029934,0.112914,0.335343,0.354057,6052.592985,31.567899,17.204530,29.105844,24.562860,29.105844
1,6.52,0.755945,19.325000,196,0.0,21.770833,29.6316,3.791667,2012,20.573593,...,0.029932,0.112904,0.335286,0.354033,6178.213349,29.570000,17.390000,29.760736,26.096551,29.760736
2,6.92,0.858993,20.262500,197,0.0,21.908333,29.3472,4.137500,2012,20.475931,...,0.029930,0.112893,0.335229,0.354008,6313.892022,31.097908,17.235624,30.299442,25.986418,30.299442
3,6.35,0.477617,16.791667,198,0.0,22.420833,28.8180,6.033333,2012,20.571045,...,0.029928,0.112882,0.335173,0.353984,6459.629005,30.868718,17.248525,28.978868,25.122209,28.978868
4,5.13,0.556820,17.016667,199,0.0,21.529167,23.1732,4.350000,2012,16.757401,...,0.029926,0.112872,0.335116,0.353960,6615.424298,30.657792,17.259663,30.653667,27.830367,30.653667


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1028 entries, 0 to 1027
Data columns (total 28 columns):
PET             1028 non-null float64
VPD             1028 non-null float64
air_temp        1028 non-null float64
doy             1028 non-null int64
precip          1028 non-null float64
soil_temp       1028 non-null float64
sw_in           1028 non-null float64
wind_speed      1028 non-null float64
year            1028 non-null int64
wp_RNET         1028 non-null float64
wp_ch4_gf       1028 non-null float64
wp_co2_gf       1028 non-null float64
wp_er           1028 non-null float64
wp_gpp          1028 non-null float64
wp_h            1028 non-null float64
wp_le           1028 non-null float64
wp_bnd1         1028 non-null float64
wp_bnd2         1028 non-null float64
wp_bnd3         1028 non-null float64
wp_bnd7         1028 non-null float64
wp_evi          1028 non-null float64
wp_lswi2        1028 non-null float64
wp_ndvi         1028 non-null float64
wp_LST.day      1028 no

In [4]:
train_cols = ["precip", "sw_in", "LW_IN.wp", "VPD", "air_temp", "PET", "wind_speed", "wp_LST.day",
             "wp_LST.night", "wp_evi", "wp_ndvi", "wp_lswi2"]
X, Y = exp.featurize(df, train_cols, ["wp_h"])
X, Y, scaler = r.preprocess(X, Y)
X.shape

(1028, 12)

In [5]:
r.random_forests_cross_val(X, Y, feature_names=train_cols)

Running Random Forests Cross Validation...
10-fold CV Acc Mean:  0.89271393183
CV Scores:  0.893832369493, 0.892573585303, 0.910750649841, 0.891182030901, 0.890299689223, 0.890038703229, 0.891748925256, 0.908928447436, 0.891296528351, 0.866488389271
OOB score: 0.89351541318
Feature Importances:
('sw_in', 0.20141875654863611)
('air_temp', 0.10438646828176758)
('PET', 0.090496371477968088)
('wp_evi', 0.089103661298346792)
('LW_IN.wp', 0.086860643195867768)
('wp_LST.night', 0.084468501895859874)
('wp_lswi2', 0.079603565487067152)
('VPD', 0.076578862069950529)
('wp_LST.day', 0.067715104060806316)
('wp_ndvi', 0.06660445851838516)
('wind_speed', 0.042647943945539994)
('precip', 0.010115663219804569)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='sqrt', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=200, n_jobs=1, oob_score=True, random_state=None,
           verbose=0, warm_start=False)

In [6]:
r.xgb_trees_cross_val(X, Y, feature_names=train_cols)

Running Gradient Boosted Trees Cross Validation...
10-fold CV Acc Mean:  0.916341764515
CV Scores:  0.91788016981, 0.930538258957, 0.930318713195, 0.918280688531, 0.917739443537, 0.914669308799, 0.90559271983, 0.910059506035, 0.909047222831, 0.909291613628
Feature Importances:
('sw_in', 0.14219476213992785)
('wind_speed', 0.10093593841177899)
('wp_lswi2', 0.09887280249941563)
('PET', 0.097781023976531975)
('VPD', 0.097465067849870121)
('LW_IN.wp', 0.091102237123308158)
('wp_evi', 0.089590837095645193)
('wp_LST.night', 0.074054545411259942)
('wp_ndvi', 0.071145726174130966)
('wp_LST.day', 0.062402471264967223)
('air_temp', 0.055272277906311923)
('precip', 0.01918231014685191)


GradientBoostingRegressor(alpha=0.9, init=None, learning_rate=0.1, loss='ls',
             max_depth=3, max_features='sqrt', max_leaf_nodes=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=200,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)

In [7]:
r.svc_cross_val(X, Y)

Running SVC Cross Validation...
10-fold CV Acc Mean:  0.848707314265
CV Scores:  0.823823283783, 0.884379784714, 0.865393368226, 0.873487164276, 0.853788120118, 0.857531148116, 0.843408794415, 0.883442830551, 0.826520915021, 0.77529773343


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [8]:
r.dnn_cross_val(X, Y)

Running Neural Network Cross Validation...
Step #1, avg. loss: 12.21579
Step #501, epoch #50, avg. loss: 1.05464
Step #1001, epoch #100, avg. loss: 0.39656
Step #1501, epoch #150, avg. loss: 0.29284
Step #2001, epoch #200, avg. loss: 0.23098
Step #2501, epoch #250, avg. loss: 0.19913
Step #3001, epoch #300, avg. loss: 0.17932
Step #3501, epoch #350, avg. loss: 0.16552
Step #4001, epoch #400, avg. loss: 0.14695
Step #4501, epoch #450, avg. loss: 0.13260
Step #1, avg. loss: 12.57497
Step #501, epoch #50, avg. loss: 1.05346
Step #1001, epoch #100, avg. loss: 0.38520
Step #1501, epoch #150, avg. loss: 0.28248
Step #2001, epoch #200, avg. loss: 0.22151
Step #2501, epoch #250, avg. loss: 0.19189
Step #3001, epoch #300, avg. loss: 0.16063
Step #3501, epoch #350, avg. loss: 0.14459
Step #4001, epoch #400, avg. loss: 0.13700
Step #4501, epoch #450, avg. loss: 0.11876
Step #1, avg. loss: 10.98234
Step #501, epoch #50, avg. loss: 1.04691
Step #1001, epoch #100, avg. loss: 0.42177
Step #1501, epoc

TensorFlowEstimator(batch_size=100, class_weight=None,
          continue_training=False, early_stopping_rounds=None,
          keep_checkpoint_every_n_hours=10000, learning_rate=0.1,
          max_to_keep=5, model_fn=<function tanh_dnn at 0x1156ae410>,
          n_classes=0, num_cores=4, optimizer='SGD', steps=5000,
          tf_master='', tf_random_seed=42, verbose=1)